In [1]:
import os
os.environ['KAGGLE_USERNAME'] = '<replace-with-your-username>'
os.environ['KAGGLE_KEY'] = '<replace-with-your-key>'
from kaggle.api.kaggle_api_extended import KaggleApi
kaggle = KaggleApi()
kaggle.authenticate()
kaggle.dataset_download_files(dataset='kamilpytlak/personal-key-indicators-of-heart-disease',
    path='archive/',
    force=True,
    quiet=False,
    unzip=True
)

100%|██████████| 21.4M/21.4M [00:04<00:00, 5.39MB/s]


In [2]:
import pandas as pd
import numpy as np

train = pd.read_csv('archive/2020/heart_2020_cleaned.csv')

numeric_features=['BMI', 'PhysicalHealth', 'MentalHealth', 'SleepTime']
categorical_features=['HeartDisease', 'Smoking', 'AlcoholDrinking', 'Stroke', 'DiffWalking', 'Sex', 'AgeCategory',
       'Race', 'Diabetic', 'PhysicalActivity', 'GenHealth','Asthma', 'KidneyDisease', 'SkinCancer']

from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
enc.fit(train[categorical_features])
train[categorical_features] = enc.transform(train[categorical_features])

y=train['HeartDisease']
train.drop('HeartDisease',axis=1,inplace=True)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(train,y,test_size=0.1,random_state=42)

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.metrics import precision_score,recall_score
from sklearn.metrics import f1_score

models = [KNeighborsClassifier(), LogisticRegression(), XGBClassifier(), ExtraTreesClassifier(),
          RandomForestClassifier(), GradientBoostingClassifier()]

from collections import namedtuple
import importlib

ClassInfo = namedtuple("ClassInfo", ["module_name", "class_name"])
models = [
    ClassInfo(module_name="sklearn.neighbors", class_name="KNeighborsClassifier"),
    ClassInfo(module_name="sklearn.linear_model", class_name="LogisticRegression"),
    ClassInfo(module_name="xgboost", class_name="XGBClassifier"),
    ClassInfo(module_name="sklearn.ensemble", class_name="ExtraTreesClassifier"),
    ClassInfo(module_name="sklearn.ensemble", class_name="RandomForestClassifier"),
    ClassInfo(module_name="sklearn.ensemble", class_name="GradientBoostingClassifier")
]
models = {class_info.class_name: class_info for class_info in models}

def instantiate_class(class_info, *args, **kwargs):
    try:
        module = importlib.import_module(class_info.module_name)
        class_ = getattr(module, class_info.class_name)
        instance = class_(*args, **kwargs)
        return instance
    except ImportError as e:
        print(f"Error importing module {class_info.module_name}: {e}")
    except AttributeError as e:
        print(f"Error getting class {class_info.class_name} from module {class_info.module_name}: {e}")
    except Exception as e:
        print(f"Error instantiating class {class_info.class_name} from module {class_info.module_name}: {e}")

for v in models.values():
    m = instantiate_class(v)
    m.fit(X_train, y_train)
    y_pred = m.predict(X_test)

    print(f'model: {str(m)}')
    print(f'Accuracy_score: {accuracy_score(y_test,y_pred)}')
    print(f'Precission_score: {precision_score(y_test,y_pred)}')
    print(f'Recall_score: {recall_score(y_test,y_pred)}')
    print(f'F1-score: {f1_score(y_test,y_pred)}')
    print('-'*30, '\n')

model: KNeighborsClassifier()
Accuracy_score: 0.9027517198248906
Precission_score: 0.3271428571428571
Recall_score: 0.0798465829846583
F1-score: 0.12836322869955158
------------------------------ 



/Users/mmortari/git/demo20240120-cvd-risk/venv/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


model: LogisticRegression()
Accuracy_score: 0.9115697310819262
Precission_score: 0.5423728813559322
Recall_score: 0.08926080892608089
F1-score: 0.1532934131736527
------------------------------ 

model: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)
Accuracy_score: 0.9114133833646029
Prec